In [24]:
import sys

sys.path.append('../')
from data_access.data_access_utility import ABRDataAccess
from classifier.classical_classifiers import SVM, RFClassifier, KNN
from utility.file_utility import FileUtility




In [40]:
cv='standard'
vector_dict=dict()
coloring_dict=dict()
feature_lists=[['snps_nonsyn_trimmed'],['gpa_trimmed','gpa_roary'],['genexp']]
for feature_list in feature_lists:
    ABRAccess=ABRDataAccess('../../amr_results/res_before_5_3_2017/intermediate_reps/',feature_list)
    vector_dict['_'.join(feature_list)]=dict()
    coloring_dict['_'.join(feature_list)]=dict()
    for drug in ABRAccess.BasicDataObj.drugs:
        vector_dict['_'.join(feature_list)][drug]=dict()
        coloring_dict['_'.join(feature_list)][drug]=dict()
        print(drug , ' features ',' and '.join(feature_list), ' Random Forest ')
        X_rep, Y, features, final_isolates = ABRAccess.get_xy_prediction_mats(drug, mapping={'0':0,'0.0':0,'1':1,'1.0':1})
        name=drug+'_'.join(feature_list)
        # for coloring
        true_label_dict={iso:Y[idx] for idx, iso in enumerate(final_isolates)}
        coloring_dict['_'.join(feature_list)][drug]=true_label_dict
        isolate_index={iso:idx for idx, iso in enumerate(final_isolates)}
        [_, _, _,_,_, _,  (pred_RF, Y1,_)]=FileUtility.load_obj('../../amr_results/5_3_2017/classifications_standard/'+name+'_RF.pickle')

        [_, _, _, _,_, _, (pred_SVM,Y2,_)]=FileUtility.load_obj('../../amr_results/5_3_2017/classifications_standard/'+name+'_SVM.pickle')

        [_, _, _, _,_, _, (pred_KNN,Y3,_)]=FileUtility.load_obj('../../amr_results/5_3_2017/classifications_standard/'+name+'_KNN.pickle')
        # vector
        vector_dict['_'.join(feature_list)][drug]={isolate:[-1 if true_label_dict[isolate]==pred_RF[isolate_index[isolate]] else 1,-1 if true_label_dict[isolate]==pred_SVM[isolate_index[isolate]] else 1 , -1 if true_label_dict[isolate]==pred_KNN[isolate_index[isolate]] else 1] for isolate in  final_isolates}

Data access created..
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/snps_nonsyn_trimmed_feature_vect.npz
Tobramycin  features  snps_nonsyn_trimmed  Random Forest 


/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KNeighborsClassifier from

Ceftazidim  features  snps_nonsyn_trimmed  Random Forest 
Ciprofloxacin  features  snps_nonsyn_trimmed  Random Forest 
Meropenem  features  snps_nonsyn_trimmed  Random Forest 
Colistin  features  snps_nonsyn_trimmed  Random Forest 
Data access created..
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/gpa_trimmed_feature_vect.npz
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/gpa_roary_feature_vect.npz
Tobramycin  features  gpa_trimmed and gpa_roary  Random Forest 
Ceftazidim  features  gpa_trimmed and gpa_roary  Random Forest 
Ciprofloxacin  features  gpa_trimmed and gpa_roary  Random Forest 
Meropenem  features  gpa_trimmed and gpa_roary  Random Forest 
Colistin  features  gpa_trimmed and gpa_roary  Random Forest 
Data access created..
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/genexp_feature_vect.npz
Tobramycin  features  genexp  Random Forest 
Ceftazidim  features  genexp  Random Forest 
Ciprofloxacin  features  genexp  Random Forest 
Meropenem

In [41]:
FileUtility.save_obj('misclassification',[vector_dict,coloring_dict])

# Create Ensembles

In [26]:
def get_res(L,S,R):
    if L==1 and S==1:
        return 1
    elif R==0:
        return 0
    return L

cv='standard'
ensamble_res=dict()
feature_lists=[['snps_nonsyn_trimmed'],['gpa_trimmed','gpa_roary'],['genexp'],['snps_nonsyn_trimmed','gpa_trimmed','gpa_roary'],['genexp','gpa_trimmed','gpa_roary'],['genexp','snps_nonsyn_trimmed'],['snps_nonsyn_trimmed','gpa_trimmed','gpa_roary','genexp']]
for feature_list in feature_lists:
    ABRAccess=ABRDataAccess('../../amr_results/res_before_5_3_2017/intermediate_reps/',feature_list)
    ensamble_res['_'.join(feature_list)]=dict()
    for drug in ABRAccess.BasicDataObj.drugs:
        ensamble_res['_'.join(feature_list)][drug]=[]
        print(drug , ' features ',' and '.join(feature_list), ' Random Forest ')
        X_rep, Y, features, final_isolates = ABRAccess.get_xy_prediction_mats(drug, mapping={'0':0,'0.0':0,'1':1,'1.0':1})
        name=drug+'_'.join(feature_list)
        try:
            [_, _, _,_,_, _,  (pred_RF, Y1,_)]=FileUtility.load_obj('../../amr_results/meeting/'+name+'_RF.pickle')
            print('not ',name, 'RF')
            [_, _, _, _,_, _, (pred_SVM,Y2,_)]=FileUtility.load_obj('../../amr_results/meeting/'+name+'_SVM.pickle')
            print('not ',name, 'SVM')
            #[_, _, _, _,_, _, (pred_KNN,Y3,_)]=FileUtility.load_obj('../../amr_results/meeting/'+name+'_KNN.pickle')
            print('not ',name, 'KNN')
            [_, _, _, _,_, _, (pred_LR,Y4,_)] =FileUtility.load_obj('../../amr_results/meeting/'+name+'_LR.pickle')
            ensamble_res['_'.join(feature_list)][drug]=([get_res(pred_LR[idx],pred_SVM[idx],pred_RF[idx]) for idx, y in enumerate(Y1)],Y)
        except:
            print('not ',name)
        
        

Data access created..
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/snps_nonsyn_trimmed_feature_vect.npz
Tobramycin  features  snps_nonsyn_trimmed  Random Forest 


/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from v

not  Tobramycinsnps_nonsyn_trimmed RF
not  Tobramycinsnps_nonsyn_trimmed SVM
not  Tobramycinsnps_nonsyn_trimmed KNN
Ceftazidim  features  snps_nonsyn_trimmed  Random Forest 
not  Ceftazidimsnps_nonsyn_trimmed RF
not  Ceftazidimsnps_nonsyn_trimmed SVM
not  Ceftazidimsnps_nonsyn_trimmed KNN
Ciprofloxacin  features  snps_nonsyn_trimmed  Random Forest 
not  Ciprofloxacinsnps_nonsyn_trimmed RF
not  Ciprofloxacinsnps_nonsyn_trimmed SVM
not  Ciprofloxacinsnps_nonsyn_trimmed KNN
Meropenem  features  snps_nonsyn_trimmed  Random Forest 
not  Meropenemsnps_nonsyn_trimmed RF
not  Meropenemsnps_nonsyn_trimmed SVM
not  Meropenemsnps_nonsyn_trimmed KNN
Colistin  features  snps_nonsyn_trimmed  Random Forest 
not  Colistinsnps_nonsyn_trimmed RF
not  Colistinsnps_nonsyn_trimmed SVM
not  Colistinsnps_nonsyn_trimmed KNN
Data access created..
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/gpa_trimmed_feature_vect.npz
@@@../../amr_results/res_before_5_3_2017/intermediate_reps/gpa_roary_feature_v

In [27]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd

def TNR(y_true, y_pred):
    '''
    :param y_true:
    :param y_pred:
    :return: True-negative rate
    '''
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_pred)):
        if (y_true[i] == y_pred[i]) and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and (y_true[i] != y_pred[i]):
            FP += 1
        if y_true[i] == y_pred[i] and y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_true[i] != y_pred[i]:
            FN += 1
    return float(TN / (TN + FP))

In [28]:
table = {'classifier': [], 'drug': [], 'feature':[],'f1_macro':[],'f1_micro':[],'precision_micro':[],'precision_macro':[],'recall_micro':[],'recall_macro':[],'accuracy':[],'tnr':[],'scores_p_0':[],'scores_r_0':[],'scores_f_1_0':[],'scores_p_1':[],'scores_r_1':[],'scores_f_1_1':[]}
for feature in list(ensamble_res.keys()):
        for drug in list(ensamble_res[feature].keys()):
            try:
                y_pred, y_true=ensamble_res[feature][drug]
                p_mic,r_mic,f_mic,s=precision_recall_fscore_support(y_true, y_pred, average='micro')
                p_mac,r_mac,f_mac,s=precision_recall_fscore_support(y_true, y_pred, average='macro')
                p_1,r_1,f_1,s=precision_recall_fscore_support(y_true, y_pred, pos_label=1)
                p_0,r_0,f_0,s=precision_recall_fscore_support(y_true, y_pred, pos_label=0)
                tnr=TNR(y_true, y_pred)
                acc=accuracy_score(y_true, y_pred)
                table['classifier'].append('Ensemble RF, KNN, SVM')
                table['drug'].append(drug)
                table['feature'].append(feature)
                table['f1_micro'].append(f_mic)
                table['f1_macro'].append(f_mac)
                table['precision_micro'].append(p_mic)
                table['precision_macro'].append(p_mac)
                table['recall_micro'].append(r_mic)
                table['recall_macro'].append(r_mac)
                table['accuracy'].append(acc)
                table['tnr'].append(tnr)
                table['scores_p_0'].append(p_0)
                table['scores_r_0'].append(r_0)
                table['scores_p_1'].append(p_1)
                table['scores_r_1'].append(r_1)
                table['scores_f_1_0'].append(f_0)
                table['scores_f_1_1'].append(f_1)
            except:
                print(feature,drug)


genexp_gpa_trimmed_gpa_roary Meropenem
genexp_gpa_trimmed_gpa_roary Tobramycin
genexp_gpa_trimmed_gpa_roary Ceftazidim
genexp_gpa_trimmed_gpa_roary Colistin
genexp_gpa_trimmed_gpa_roary Ciprofloxacin
genexp_snps_nonsyn_trimmed Meropenem
genexp_snps_nonsyn_trimmed Tobramycin
genexp_snps_nonsyn_trimmed Ceftazidim
genexp_snps_nonsyn_trimmed Colistin
genexp_snps_nonsyn_trimmed Ciprofloxacin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary_genexp Meropenem
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary_genexp Tobramycin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary_genexp Ceftazidim
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary_genexp Colistin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary_genexp Ciprofloxacin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary Meropenem
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary Tobramycin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary Ceftazidim
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary Colistin
snps_nonsyn_trimmed_gpa_trimmed_gpa_roary Ciprofloxacin


/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/metrics/classification.py:1045: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [29]:
df=pd.DataFrame(data=table,columns=['drug', 'feature','classifier','f1_macro','scores_r_0','f1_micro','accuracy','tnr','precision_micro','precision_macro','recall_micro','recall_macro','scores_p_0','scores_r_0','scores_f_1_0','scores_p_1','scores_r_1','scores_f_1_1'])

In [30]:
writer = pd.ExcelWriter('ensemble.xlsx')
df.to_excel(writer)
writer.save()